### 📦 **Libraries Used for EDA _IMDB_ 🎬**  

In [5]:
import os
import sqlite3
import pandas as pd
from lets_plot import *
LetsPlot.setup_html()
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

### 🗄️ **Creating a Database for Movie Data 🎬** 

In [6]:
conn = sqlite3.connect("../data/clean/IMDB.db")

# 💰 Showing **Profit** Analysis 📊

In [7]:
query = """
SELECT 
    pc.production_company,
    SUM(im.budget) AS total_budget,
    SUM(im.gross_in_million) AS total_gross,
    SUM(im.gross_in_million) - SUM(im.budget) AS profit_loss
FROM imdb_movies im
JOIN production_companies pc ON im.movie_id = pc.movie_id
WHERE im.gross_in_million > 0
GROUP BY pc.production_company
ORDER BY profit_loss DESC
LIMIT 15;  
"""
df5 = pd.read_sql_query(query, conn)

df5['status'] = df5['profit_loss'].apply(lambda x: 'Profit' if x > 0 else 'Loss')
df5['color'] = df5['status'].map({'Profit': 'blue', 'Loss': 'red'})  

p = ggplot(df5, aes(x='production_company', y='profit_loss', fill='status')) + \
    geom_bar(stat='identity', show_legend=True) + \
    geom_text(aes(label='profit_loss'), size=8, vjust=-0.3, color='black') + \
    ggtitle("Profit of Top Production Companies") + \
    ylab("Profit in Million") + \
    scale_fill_manual(values={'Profit': 'blue', 'Loss': 'red'}) + \
    theme_minimal() + \
    theme(
        axis_text_x=element_text(size=10, angle=45),  
        plot_title=element_text(size=14, face="bold")
    ) + \
    ggsize(4000, 1500)
p

In [8]:
# Best production companies based on total profit
# Highest-rated production companies
# Biggest spenders on movie production
# Top-grossing production companies
# International reach of production companies
# Revenue trends over time
# Genre distribution across production companies
# Bubble charts for multiple financial metrics

## 🏆 Best **Production Companies** Based on Total **Profit 💰**  

In [9]:
# import plotly.express as px
# import pandas as pd

query = """
SELECT pc.production_company, 
       SUM(im.gross_in_million - im.budget) AS total_profit
FROM imdb_movies im
JOIN production_companies pc ON im.movie_id = pc.movie_id
WHERE im.budget IS NOT NULL AND im.gross_in_million IS NOT NULL
GROUP BY pc.production_company
ORDER BY total_profit DESC
LIMIT 20;
"""

df = pd.read_sql_query(query, conn)

df = df.sort_values(by="total_profit", ascending=True)

fig = px.scatter(df, x="total_profit", y="production_company", 
                 size=[8]*len(df), color="production_company", 
                 title="Top Production Companies by Total Profit",
                 labels={"total_profit": "Total Profit (in Million $)", "production_company": "Production Company"})

for i, row in df.iterrows():
    fig.add_shape(type="line",
                  x0=0, x1=row["total_profit"],
                  y0=row["production_company"], y1=row["production_company"],
                  line=dict(color="blue", width=2))

fig.update_layout(showlegend=False, height=800, width=1200)

fig.show()


# 🔹Production Companies **Loss Data 📉**  

In [10]:
query = """
SELECT pc.production_company, 
       SUM(im.budget) AS total_budget, 
       SUM(im.gross_in_million) AS total_revenue,
       SUM(im.budget) - SUM(im.gross_in_million) AS total_loss
FROM imdb_movies im
JOIN production_companies pc ON im.movie_id = pc.movie_id
WHERE im.budget IS NOT NULL 
AND im.gross_in_million IS NOT NULL
AND pc.production_company IS NOT NULL  
AND pc.production_company != ''  
AND LOWER(pc.production_company) != 'none'  
GROUP BY pc.production_company
HAVING total_loss > 0  
ORDER BY total_loss DESC  
LIMIT 20;  -- Show top 20 production companies with the biggest losses
"""
df = pd.read_sql_query(query, conn)

fig = px.pie(df, 
             names="production_company", 
             values="total_loss", 
             title="Losses by Production Company",
             hole=0.4,  
             color="total_loss",  
             color_discrete_sequence=px.colors.sequential.Greens)

fig.update_traces(textinfo="label+value", 
                  textfont_size=14, 
                  marker=dict(line=dict(color="black", width=1)))

fig.update_layout(width=1200, 
                  height=800, 
                  showlegend=True)

fig.show()

# ⭐ **Top Highest-Rated Production Companies 🎬**  

In [11]:
query = """
SELECT pc.production_company, AVG(im.rating) AS avg_rating
FROM imdb_movies im
JOIN production_companies pc ON im.movie_id = pc.movie_id
WHERE im.rating IS NOT NULL
GROUP BY pc.production_company
ORDER BY avg_rating DESC
LIMIT 20;
"""
df = pd.read_sql_query(query, conn)

df = df.sort_values(by="avg_rating", ascending=False)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df["production_company"], 
    y=df["avg_rating"], 
    mode="lines", 
    line=dict(color="skyblue", width=2),
    hoverinfo="x+y+text",
    text=["Average Rating: " + str(rating) for rating in df["avg_rating"]]
))

fig.add_trace(go.Scatter(
    x=df["production_company"], 
    y=df["avg_rating"], 
    mode="markers", 
    marker=dict(color="red", size=10),
    hoverinfo="x+y+text",
    text=["Average Rating: " + str(rating) for rating in df["avg_rating"]]
))

fig.update_layout(
    title="Top Highest-Rated Production Companies",
    xaxis_title="Production Company",
    yaxis_title="Average Rating",
    yaxis=dict(range=[5, 10], tickvals=[5, 7, 9, 10]),
    xaxis_tickangle=90,
    template="plotly_white",
    hovermode="closest",
    height=800,
)

fig.show()

# 💰 Biggest Spenders on **Movie Production 🎬** 

In [12]:
query = """
SELECT pc.production_company, 
       SUM(im.budget) AS total_budget
FROM imdb_movies im
JOIN production_companies pc ON im.movie_id = pc.movie_id
WHERE im.budget IS NOT NULL 
AND pc.production_company IS NOT NULL  
AND pc.production_company != ''  
AND LOWER(pc.production_company) != 'none'  -- Exclude string "None"
GROUP BY pc.production_company
ORDER BY total_budget DESC
LIMIT 20;
"""
df = pd.read_sql_query(query, conn)

df = df.sort_values(by="total_budget", ascending=True)

p = ggplot(df, aes(x="total_budget", y="production_company", fill="total_budget")) + \
    geom_bar(stat="identity", show_legend=False) + \
    labs(title="Biggest Spenders on Movie Production", 
         x="Total Budget (in Million $)", 
         y="Production Company") + \
    scale_fill_gradient(low="lightblue", high="darkblue") + \
    theme_minimal() + \
    ggsize(1200, 800)

p.show()

# 💰 **Top-Grossing Production Companies 🎬**  

In [13]:
query = """
SELECT pc.production_company, 
       SUM(im.gross_in_million) AS total_gross
FROM imdb_movies im
JOIN production_companies pc ON im.movie_id = pc.movie_id
WHERE im.gross_in_million IS NOT NULL  
AND pc.production_company IS NOT NULL  
AND pc.production_company != ''  
AND LOWER(pc.production_company) != 'none'  -- Exclude string "None"
GROUP BY pc.production_company
ORDER BY total_gross DESC
LIMIT 20;
"""
df = pd.read_sql_query(query, conn)

df = df.sort_values(by="total_gross", ascending=True)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df["total_gross"],
    y=df["production_company"],
    mode='lines',
    line=dict(color='skyblue', width=2),
    showlegend=True
))

fig.add_trace(go.Scatter(
    x=df["total_gross"],
    y=df["production_company"],
    mode='markers',
    marker=dict(color='red', size=10),
    name='Total Gross'
))

fig.update_layout(
    title="Top-Grossing Production Companies (Lollipop Chart)",
    xaxis_title="Total Gross Revenue (in Million $)",
    yaxis_title="Production Company",
    template="plotly_white",
    width=1200,
    height=800
)
fig.show()

# 📈 **Revenue Trends Over Time 💰**  


In [14]:
query = """
SELECT im.release_year, 
       pc.production_company, 
       SUM(im.gross_in_million) AS total_revenue,
       COUNT(im.movie_id) AS movie_count  -- Count movies per production company
FROM imdb_movies im
JOIN production_companies pc ON im.movie_id = pc.movie_id
WHERE im.gross_in_million IS NOT NULL  
AND im.release_year BETWEEN 2000 AND 2026  -- Year range
AND pc.production_company IS NOT NULL  
AND pc.production_company != ''  
AND LOWER(pc.production_company) != 'none'  -- Exclude "None" as a string
GROUP BY im.release_year, pc.production_company
ORDER BY im.release_year;
"""
df = pd.read_sql_query(query, conn)

fig = px.scatter(df, 
                 x="release_year", 
                 y="total_revenue", 
                 size="movie_count",  
                 color="production_company",  
                 animation_frame="release_year",  
                 hover_name="production_company",  
                 title="Revenue Trends by Production Company ",
                 labels={"total_revenue": "Total Revenue (Million $)", "release_year": "Year"},
                 size_max=100,  
                 template="plotly_white")

fig.update_layout(
    xaxis_title="Release Year",
    yaxis_title="Total Revenue (in Million $)",
    width=1100,
    height=600,
    xaxis=dict(tickmode="linear", tickangle=45),
        sliders=[dict(
        transition={"duration": 500},
        currentvalue={"prefix": "Year: "}
    )]
)

fig.show()


# 📈**Revenue Trends of Production Companies(Financial Data) 💰**  

In [15]:
query = """
SELECT im.release_year, 
       pc.production_company, 
       SUM(im.gross_in_million) AS total_revenue,
       SUM(im.budget) AS total_budget,
       SUM(im.gross_in_million) - SUM(im.budget) AS profit,  -- Profit Calculation
       COUNT(im.movie_id) AS movie_count  -- Count movies per production company
FROM imdb_movies im
JOIN production_companies pc ON im.movie_id = pc.movie_id
WHERE im.gross_in_million IS NOT NULL AND im.budget IS NOT NULL
AND im.release_year BETWEEN 2000 AND 2026  -- Year range
AND pc.production_company IS NOT NULL  
AND pc.production_company != ''  
AND LOWER(pc.production_company) != 'none'  -- Exclude "None" as a string
GROUP BY im.release_year, pc.production_company
ORDER BY im.release_year;
"""
df = pd.read_sql_query(query, conn)

all_years = pd.DataFrame({"release_year": list(range(2005, 2027))})  
df = all_years.merge(df, on="release_year", how="left").fillna({
    "total_revenue": 0, "profit": 0, "total_budget": 0, "movie_count": 0
})

fig = px.scatter(df, 
                 x="release_year", 
                 y="total_revenue", 
                 size="movie_count", 
                 color="production_company",
                 title="Revenue Trends by Production Company ",
                 labels={"total_revenue": "Total Revenue (Million $)", "release_year": "Year"},
                 width=1200,  
                 height=700   )

dropdown_buttons = [
    {"label": "All", "method": "update", "args": [{"visible": [True] * len(df.production_company.unique())}]}
]

for company in df["production_company"].unique():
    dropdown_buttons.append({
        "label": company,
        "method": "update",
        "args": [{"visible": df["production_company"] == company}]
    })

fig.update_layout(
    updatemenus=[{
        "buttons": dropdown_buttons,
        "direction": "down",
        "showactive": True,
        "x": 0.17,  
        "y": 1.2,   
        "xanchor": "left",
        "yanchor": "top",
        "font": {"size": 12}  
    }]
)

fig.show()


# 🎬 **Director's Contribution to Total Revenue 💰**  

In [16]:
df_movies = pd.read_sql("SELECT director, rating, gross_in_million FROM imdb_movies", conn)


df_movies = df_movies[df_movies["gross_in_million"] > 0]

fig_treemap = px.treemap(df_movies, 
                          path=["director"], 
                          values="gross_in_million", 
                          color="rating",  
                          title="Director's Contribution to Total Revenue",
                          labels={"gross_in_million": "Total Revenue (Million $)", "director": "Director", "rating": "IMDb Rating"},
                          height=700,
                          width=1200)

fig_treemap.show()


# 🌍 **Country-wise Movie Revenue Distribution 💰**  

In [17]:
df_countries = pd.read_sql("SELECT c.countries, m.gross_in_million FROM countries c JOIN imdb_movies m ON c.movie_id = m.movie_id", conn)
fig_country = px.box(df_countries, 
                     x="countries", 
                     y="gross_in_million",
                     color="countries",  
                     title="Country-wise Movie Revenue Distribution",
                     labels={"gross_in_million": "Revenue (Million $)", "countries": "Country"},
                     points="all",
                    height=700,
                     width=1200) 

fig_country.show()

# 🎬 **Movie Ratings by Language 🌎**  

In [18]:
df_languages = pd.read_sql("SELECT l.language, m.rating FROM language l JOIN imdb_movies m ON l.movie_id = m.movie_id", conn)
fig_language = px.violin(df_languages, x="language", y="rating",
                         title="Movie Ratings by Language",
                         labels={"rating": "IMDb Rating", "language": "Language"},
                         box=True, points="all",
                         height=700,
                         width=1500)
fig_language.show()
